## Final Project, Part 1-2:


#### Problem statement

millions of transactions payments to sellers are made everyday on ecommerce platform like shopee or lazada. with the increase in volume, an intelligent system of fraud detection is required for the employees to deal with the mounting transactions volume.

Questions: 
Q1: how to  predict if a tranction is a fraud,using a selected list of given attributed
Q2: is the classification model better than the current Flagging system that flagged transactions that are more than 200,000?

So far 18 out of 18 flagged transactions are indeed Fraud
but 8200 fraud transactions are NOT flagged.

Attribute CO
Target Columns: 'isFraud'

Objective:
Build a Classification Model that can predict if a transaction is fraud or not fraud
    - model 1: Classification model using Linear Regression
    - model 2: Logistic Regression
    - model 3: decision tree?
    
Determine if the Classification model is more accurate compared to the baseline:
    - the baseline is the current flagged of fraud

#### Data Source

https://www.kaggle.com/ntnu-testimon/paysim1

#### Risks or limitations
- Curse of dimentionality:
The dataset might be too thin 6m rows and
- imbalance dataset, the non-fraud observations might be over presented
- unknown variables that are not given in the orginal dataset that might have predicting power over whether a transaction is fraud

### Description of Given Dataset

1. step: Maps a unit of time in the real world. In this case 1 step is 1 hour of time.
2. type: CASH-IN, CASH-OUT, DEBIT, PAYMENT and TRANSFER
3. amount: amount of the transaction in local currency
4. nameOrig: customer who started the transaction
5. oldbalanceOrg: initial balance before the transaction
6. newbalanceOrig: customer's balance after the transaction.
7. nameDest: recipient ID of the transaction. 
8. oldbalanceDest: initial recipient balance before the transaction.
9. newbalanceDest: recipient's balance after the transaction.
10. isFraud: identifies a fraudulent transaction (1) and non fraudulent (0)
11. ifFlaggedFraud: flags illegal attempts to transfer more than 200.000 in a single transaction. 


#### Detailed Steps Taken:
1. from sklearn.preprocessing import StandardScaler
2. from sklearn.preprocessing import LabelEncoder
3. smote
3. train, test, split
4. .fit

5. sklearn.metrics import classification_report
    - the row with the lower support col is the minority class
    - Recal(sensitity) is the ratio of correctly predicted positive to th all observation 
7. calculate the Recall of the minority class, should definitely zoom in when do the classification report
8. might need to sacrifice some precision for recall, because the cost of not detecting a fraud case is very high

9. can adjust threshold in logistic regression (0.5 to 0.3) to adjust for the cost: if the cost of true positive is higher than false positive (adjust threhold is not frequent)

> EDA

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [33]:
path_to_file = 'C:\\Users\\harriet.ly\\Documents\\General Assembly Notes\\GA_Projects\\Projects\\Project 4\\fraud.csv'
dataset = pd.read_csv(path_to_file)
dataset.head(3)

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0


In [37]:
# check number of rows and columns
dataset.shape

(6362620, 10)

In [44]:
# Test if there is any null values
dataset.isnull().values.any()

False

In [45]:
#1. Check if there are repeated nameOrig and nameDest, if there are alot, we will use label encode them to use in our model, 
# otherwise, we drop them
dataset['nameOrig'].nunique() # check if there are nameOrig with multiple transactions

6353307

In [46]:
dataset['nameDest'].nunique() # check if there are repeated nameDest

2722362

In [47]:
# There are a lot fewer unique values for nameDest, meaning lots of repeated nameDest, hence we will proceed to encode nameDest 
# to use as an attribute in our model, and drop 'nameOrig' as it is considered as unique as an ID

In [ ]:
# Label encoder on nameOrig and nameDest, these two variables are like IDs of the transactions but we will labelencode them to
# include them into the model because they may contain
# important information on the pattern/frequence of the same nameDest or nameOrg. 
# for isntance, highly repeated nameDest is an indicator of fraud

from sklearn import preprocessing

#instantiate
le = preprocessing.LabelEncoder()

#Labelencode nameorig, this process might take a while to run
le.fit(dataset['nameDest'])

#nameOrigin_Encoded=le.transform(dataset['nameOrig'])
nameDest_Encoded=le.transform(dataset['nameDest'])

#dataset['nameOrigin_Encoded']=nameOrigin_Encoded
dataset['nameDest_Encoded']=nameDest_Encoded

In [51]:
dataset.head().sort_values(by='nameDest_Encoded',ascending=True)

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,error_old_new_balanceOrig,error_old_new_balanceDest,nameDest_Encoded
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0.000000e+00,21001.00,391696
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0.000000e+00,-181.00,439685
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0.000000e+00,-11668.14,828919
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,1.455192e-11,-9839.64,1662094
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,-1.136868e-12,-1864.28,1733924


In [35]:
#2. Check if col isFlaggedFraud is completed relatd to Col amount. If amount > 200k, isFlaggedFraud =1
data_flaggedFraud = dataset[dataset['isFlaggedFraud']==1]
data_flaggedFraud[data_flaggedFraud['amount']<200000].shape

(0, 11)

In [36]:
# we will omit column isFlaggFraud as it is completed related to col amount, otherwise the amount attribute will be 
# over presented in our model
dataset=dataset.drop(columns='isFlaggedFraud')
dataset.head(3)

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1


In [40]:
#3. Next, we check if the Col amount is equal to the abs(newbalanceOrig-oldbalanceOrg), In a genuine transaction, the difference between the
# beginning and ending balance of the original account should be the same as the amount of transactions.
# Note that oldbalanceOrg is not necessary always larger than newbalanceOrig, depending on what kind of transactions, hence we use abs()
dataset['error_old_new_balanceOrig']= abs(dataset['oldbalanceOrg'] - dataset['newbalanceOrig'])-dataset['amount']
dataset[dataset['error_old_new_balanceOrig']>1].shape

(0, 12)

In [ ]:
# there are 0 transactions whose amount do not tally with the origin beginning and ending balances, hence 
# 'error_old_new_balanceOrigin' should be used as an attribute in our fraud detection model

In [41]:
#4. Next, we check if the Col amount is equal to the abs(newbalanceDest-oldbalanceDest)
dataset['error_old_new_balanceDest']= abs(dataset['newbalanceDest']-dataset['oldbalanceDest'])-dataset['amount']
dataset[dataset['error_old_new_balanceDest']>1].shape

(258456, 12)

In [ ]:
# there are 258k+ transactions whose amount do not tally with the destination beginning and ending balances, hence 
# 'error_old_new_balanceDest' should be used as an attribute in our fraud detection model

In [73]:
#5. Next we check for symmetry if the change in destination account is the same as the change in the original account
dataset['asymmetry']= abs(dataset['newbalanceDest']-dataset['oldbalanceDest']) != abs(dataset['oldbalanceOrg'] - dataset['newbalanceOrig'])
dataset[dataset['isFraud']==1]['asymmetry'].sum()

6036

In [ ]:
# there is a very high percentage of fraud case, 6000 out of 8000 case of fraud has the asymmetry pattern, 
# therefore we will include the asymmetry in our fraud detection model
# although we now need to be extra careful of multicollinearity in our model if correlated attributes are present
# we will look into removing at least one of the 4 attributes: newbalanceDest, oldbalanceDest, oldbalanceOrg, newbalanceOrig

In [75]:
#6. Check on the categorical attribute 'type'
dataset['type'].unique()

array(['PAYMENT', 'TRANSFER', 'CASH_OUT', 'DEBIT', 'CASH_IN'],
      dtype=object)

In [83]:
#to use get_dummies to transform this categorical col into 5 Cols that take values 0 and 1
dataset=pd.get_dummies(data=dataset, columns=['type'])
dataset.head()

,step,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,error_old_new_balanceOrig,error_old_new_balanceDest,nameDest_Encoded,symmetry,asymmetry,type_CASH_IN,type_CASH_OUT,type_DEBIT,type_PAYMENT,type_TRANSFER
0,1,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,1.455192e-11,-9839.64,1662094,False,True,0,0,0,1,0
1,1,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,-1.136868e-12,-1864.28,1733924,False,True,0,0,0,1,0
2,1,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0.000000e+00,-181.00,439685,False,True,0,0,0,0,1
3,1,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0.000000e+00,21001.00,391696,False,True,0,1,0,0,0
4,1,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0.000000e+00,-11668.14,828919,False,True,0,0,0,1,0


In [86]:
# Lastly in the EDA/preliminary feature selection, we drop attributes columns that are correlated to other attributes
dataset.drop(['nameOrig','nameDest'], axis=1,inplace=True)
dataset.head()

,step,amount,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,error_old_new_balanceOrig,error_old_new_balanceDest,nameDest_Encoded,symmetry,asymmetry,type_CASH_IN,type_CASH_OUT,type_DEBIT,type_PAYMENT,type_TRANSFER
0,1,9839.64,170136.0,160296.36,M1979787155,0.0,0.0,0,1.455192e-11,-9839.64,1662094,False,True,0,0,0,1,0
1,1,1864.28,21249.0,19384.72,M2044282225,0.0,0.0,0,-1.136868e-12,-1864.28,1733924,False,True,0,0,0,1,0
2,1,181.00,181.0,0.00,C553264065,0.0,0.0,1,0.000000e+00,-181.00,439685,False,True,0,0,0,0,1
3,1,181.00,181.0,0.00,C38997010,21182.0,0.0,1,0.000000e+00,21001.00,391696,False,True,0,1,0,0,0
4,1,11668.14,41554.0,29885.86,M1230701703,0.0,0.0,0,0.000000e+00,-11668.14,828919,False,True,0,0,0,1,0


> Data Processing

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,error_old_new_balanceOrig,error_old_new_balanceDest,nameDest_Encoded
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0.000000e+00,21001.00,391696
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0.000000e+00,-181.00,439685
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0.000000e+00,-11668.14,828919
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,1.455192e-11,-9839.64,1662094
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,-1.136868e-12,-1864.28,1733924


In [49]:
dataset[dataset['isFlaggedFraud']==1]

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud,nameOrigin_Encoded
2736446,212,TRANSFER,4953893.08,C728984460,4953893.08,4953893.08,C639921569,0.0,0.0,1,1,5462026
3247297,250,TRANSFER,1343002.08,C1100582606,1343002.08,1343002.08,C1147517658,0.0,0.0,1,1,329539
3760288,279,TRANSFER,536624.41,C1035541766,536624.41,536624.41,C1100697970,0.0,0.0,1,1,116746
5563713,387,TRANSFER,4892193.09,C908544136,4892193.09,4892193.09,C891140444,0.0,0.0,1,1,6052892
5996407,425,TRANSFER,10000000.00,C689608084,19585040.37,19585040.37,C1392803603,0.0,0.0,1,1,5332564
5996409,425,TRANSFER,9585040.37,C452586515,19585040.37,19585040.37,C1109166882,0.0,0.0,1,1,4552191
6168499,554,TRANSFER,3576297.10,C193696150,3576297.10,3576297.10,C484597480,0.0,0.0,1,1,3078566
6205439,586,TRANSFER,353874.22,C1684585475,353874.22,353874.22,C1770418982,0.0,0.0,1,1,2248279
6266413,617,TRANSFER,2542664.27,C786455622,2542664.27,2542664.27,C661958277,0.0,0.0,1,1,5651751
6281482,646,TRANSFER,10000000.00,C19004745,10399045.08,10399045.08,C1806199534,0.0,0.0,1,1,2958273
